# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-06-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-05-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-06 05:23:55,33.93911,67.709953,55265,2407,48018,4840.0,Afghanistan,141.965978,4.355379
1,NaN,NaN,NaN,Albania,2021-02-06 05:23:55,41.15330,20.168300,83082,1433,50219,31430.0,Albania,2886.997012,1.724802
2,NaN,NaN,NaN,Algeria,2021-02-06 05:23:55,28.03390,1.659600,108629,2909,74330,31390.0,Algeria,247.722728,2.677922
3,NaN,NaN,NaN,Andorra,2021-02-06 05:23:55,42.50630,1.521800,10172,104,9444,624.0,Andorra,13165.081214,1.022414
4,NaN,NaN,NaN,Angola,2021-02-06 05:23:55,-11.20270,17.873900,20030,474,18431,1125.0,Angola,60.943944,2.366450


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2389,2397,2399,2400,2400,2404,2405,2406,2407,2407
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1339,1350,1358,1369,1380,1393,1398,1404,1418,1433
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2877,2881,2884,2888,2891,2894,2898,2900,2904,2909


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,47549,47583,47606,47609,47679,47723,47798,47982,47995,48018
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,45383,45906,46420,46927,47424,47922,48377,48958,49543,50219
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,72538,72745,72956,73152,73344,73530,73732,73940,74136,74330


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5499,5554,5596,5596,5669,5683,5723,5753,5811,5824
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,17629,17779,17922,17922,18126,18211,18344,18418,18494,18568
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1898,1920,1931,1931,1951,1956,1966,1981,1989,1994


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,67,69,69,69,69,69,70,71,74,76
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,217,225,224,224,224,224,228,228,233,234
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,40,40,40,40,40,40,40,42,43,44


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-02-06 05:23:55,32.539527,-86.644082,5824,76,0,5748.0,"Autauga, Alabama, US",10424.385616,1.304945
688,1075.0,Lamar,Alabama,US,2021-02-06 05:23:55,33.779950,-88.096680,1264,30,0,1234.0,"Lamar, Alabama, US",9156.102861,2.373418
689,1077.0,Lauderdale,Alabama,US,2021-02-06 05:23:55,34.901719,-87.656247,8332,179,0,8153.0,"Lauderdale, Alabama, US",8985.322822,2.148344


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,26813772,459555,0
India,10814304,154918,10510796
Brazil,9447165,230034,8396148


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,26813772,459555,0,26354217,2021-02-06 05:23:55,37.936303,-91.379001
India,10814304,154918,10510796,148590,2021-02-06 05:23:55,23.088275,81.806127
Brazil,9447165,230034,8396148,820983,2021-02-06 05:23:55,-12.669522,-48.480493
United Kingdom,3922910,111477,10094,3801339,2021-02-06 05:23:55,30.744028,-35.367255
Russia,3891274,74520,3375999,440755,2021-02-06 05:23:55,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3399745,43690,0
Texas,2478003,38893,0
Florida,1763873,27457,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3399745,43690,0,3356055,2021-02-06 05:23:55,37.843962,-120.728594
Texas,2478003,38893,0,2439110,2021-02-06 05:23:55,31.660643,-98.653069
Florida,1763873,27457,0,1736416,2021-02-06 05:23:55,28.940755,-82.700744
New York,1458809,44466,0,1414343,2021-02-06 05:23:55,42.544151,-75.474183
Illinois,1141219,21603,0,1119616,2021-02-06 05:23:55,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1138970,17772,0
Arizona,Maricopa,483775,7885,0
Illinois,Cook,458600,8977,0
Florida,Miami-Dade,380716,4964,0
Texas,Harris,326248,4441,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1138970,17772,0,1121198,2021-02-06 05:23:55,34.308284,-118.228241,6037.0
Arizona,Maricopa,483775,7885,0,475890,2021-02-06 05:23:55,33.348359,-112.491815,4013.0
Illinois,Cook,458600,8977,0,449623,2021-02-06 05:23:55,41.841448,-87.816588,17031.0
Florida,Miami-Dade,380716,4964,0,375752,2021-02-06 05:23:55,25.611236,-80.551706,12086.0
Texas,Harris,326248,4441,0,321807,2021-02-06 05:23:55,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,55174,80941,108116,10070,19937,249,1952744,167421,28838,418283,...,3882972,43215,78859,1,128315,1948,160426,2122,57489,33964
2021-02-04,55231,81993,108381,10137,19996,268,1961635,167568,28842,419801,...,3903706,43804,78916,1,128775,1957,161087,2122,59003,34171
2021-02-05,55265,83082,108629,10172,20030,277,1970009,167726,28848,421189,...,3922910,44303,78989,1,129231,1976,161559,2124,60427,34331


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,2406,1404,2900,103,470,7,48539,3096,909,7902,...,109547,464,621,0,1209,35,1857,615,804,1269
2021-02-04,2407,1418,2904,103,473,7,48700,3107,909,7936,...,110462,476,621,0,1216,35,1865,615,812,1288
2021-02-05,2407,1433,2909,104,474,7,48985,3109,909,7973,...,111477,478,621,0,1223,35,1867,615,828,1303


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,47982,48958,73940,9313,18335,179,1745208,158693,26079,396709,...,9936,36820,77333,1,120204,1461,150663,1427,50479,27391
2021-02-04,47995,49543,74136,9395,18381,181,1754705,159045,26080,397910,...,10011,37411,77368,1,120644,1465,151082,1427,51305,27759
2021-02-05,48018,50219,74330,9444,18431,183,1763762,159350,26083,399329,...,10094,37967,77443,1,121094,1465,151690,1427,52045,28145


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5499,5554,5596,5596,5669,5683,5723,5753,5811,5824
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1220,1230,1235,1235,1248,1249,1251,1253,1260,1264
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8049,8088,8134,8134,8180,8191,8216,8255,8298,8332
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2597,2616,2628,2628,2647,2654,2665,2682,2699,2708
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,13482,13602,13679,13679,13829,13878,13958,14048,14149,14197


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-03,465056,54820,0,767379,300430,3368281,399267,257004,78982,49,...,733216,2449535,350000,12329,2430,513339,316294,122470,595756,52288
2021-02-04,467823,55005,0,771796,302899,3382932,400851,257941,79225,49,...,736370,2463424,351273,12503,2440,516398,317878,123044,597475,52468
2021-02-05,469319,55259,0,775622,304723,3399745,402754,259372,79546,49,...,739031,2478003,352489,12612,2449,521467,319371,123641,598902,52576


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-03        5753   18418    1981  2321   5768    1093   1838   12179   
2021-02-04        5811   18494    1989  2327   5842    1107   1847   12253   
2021-02-05        5824   18568    1994  2331   5871    1113   1853   12325   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-03         3233     1718  ...    2549    560     2905      659   
2021-02-04         3239     1723  ...    2549    558     2925      662   
2021-02-05         3249     1724  ...    2546    560     2933      665   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-03           3519  3161  1981          0      869    612  
2021-02-04           3529  3199  1984          0      870    613  
2021-02-05           3533  3219  1984          0      870    613  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-03,8203,279,0,13576,4985,42484,5664,7157,1130,0,...,10033,38039,1697,179,24,6575,4388,2058,6496,624
2021-02-04,8365,279,0,13752,5009,43120,5682,7185,1135,0,...,10202,38453,1711,181,24,6650,4416,2080,6545,624
2021-02-05,8449,279,0,13948,5050,43690,5705,7214,1191,0,...,10405,38893,1728,181,24,6732,4449,2100,6575,624


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-03          71     228      42   52    105      29     60     239   
2021-02-04          74     233      43   52    109      29     62     242   
2021-02-05          76     234      44   52    111      29     62     244   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-03           80       36  ...      23     11       22        7   
2021-02-04           81       36  ...      23     11       22        7   
2021-02-05           81       36  ...      23     11       22        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-03             33     7    12          0       26      4  
2021-02-04             33     7    12          0       26      5  
2021-02-05             33     7    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,0.000962,0.012598,0.002550,0.005291,0.001859,0.000000,0.004732,0.001136,0.000312,0.003647,...,0.004973,0.012844,0.000736,0.0,0.004407,0.035069,0.002938,0.000000,0.022336,0.004436
2021-02-04,0.001033,0.012997,0.002451,0.006653,0.002959,0.076305,0.004553,0.000878,0.000139,0.003629,...,0.005340,0.013630,0.000723,0.0,0.003585,0.004620,0.004120,0.000000,0.026335,0.006095
2021-02-05,0.000616,0.013282,0.002288,0.003453,0.001700,0.033582,0.004269,0.000943,0.000208,0.003306,...,0.004919,0.011392,0.000925,0.0,0.003541,0.009709,0.002930,0.000943,0.024134,0.004682


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,0.000416,0.004292,0.000690,0.009804,0.004274,0.0,0.002333,0.002266,0.0,0.007009,...,0.012215,0.024283,0.0,NaN,0.005824,0.0,0.004327,0.0,0.012594,0.011962
2021-02-04,0.000416,0.009972,0.001379,0.000000,0.006383,0.0,0.003317,0.003553,0.0,0.004303,...,0.008353,0.025862,0.0,NaN,0.005790,0.0,0.004308,0.0,0.009950,0.014972
2021-02-05,0.000000,0.010578,0.001722,0.009709,0.002114,0.0,0.005852,0.000644,0.0,0.004662,...,0.009189,0.004202,0.0,NaN,0.005757,0.0,0.001072,0.0,0.019704,0.011646


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,0.003850,0.012010,0.002821,0.006593,0.005649,0.000000,0.003704,0.002261,0.000345,0.003524,...,0.012019,0.017746,0.000608,0.0,0.004404,0.000685,0.003597,0.0,0.015756,0.022281
2021-02-04,0.000271,0.011949,0.002651,0.008805,0.002509,0.011173,0.005442,0.002218,0.000038,0.003027,...,0.007548,0.016051,0.000453,0.0,0.003660,0.002738,0.002781,0.0,0.016363,0.013435
2021-02-05,0.000479,0.013645,0.002617,0.005216,0.002720,0.011050,0.005162,0.001918,0.000115,0.003566,...,0.008291,0.014862,0.000969,0.0,0.003730,0.000000,0.004024,0.0,0.014424,0.013905


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-03,0.004575,0.004140,NaN,0.003001,0.008141,0.003725,0.003090,0.001879,0.003634,0.0,...,0.002538,0.007340,0.004566,0.010905,0.001649,0.005798,0.005091,0.004388,0.002590,0.003069
2021-02-04,0.005950,0.003375,NaN,0.005756,0.008218,0.004350,0.003967,0.003646,0.003077,0.0,...,0.004302,0.005670,0.003637,0.014113,0.004115,0.005959,0.005008,0.004687,0.002885,0.003442
2021-02-05,0.003198,0.004618,NaN,0.004957,0.006022,0.004970,0.004747,0.005548,0.004052,0.0,...,0.003614,0.005918,0.003462,0.008718,0.003689,0.009816,0.004697,0.004852,0.002388,0.002058


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-03      0.005242  0.004034  0.007630  0.000862  0.004003  0.005520   
2021-02-04      0.010082  0.004126  0.004038  0.002585  0.012829  0.012809   
2021-02-05      0.002237  0.004001  0.002514  0.001719  0.004964  0.005420   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-03      0.002728  0.006363  0.004349  0.002334  ...  0.002359   
2021-02-04      0.004897  0.006076  0.001856  0.002910  ...  0.000000   
2021-02-05      0.003249  0.005876  0.003087  0.000580  ... -0.001177   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-03      0.001789  0.000689  0.006107   0.002564  0.003174  0.003038   
2021-02-04     -0.003571  0.006885  0.004552   0.002842  0.012022  0.001514   
2021-02-05      0.003584  0.002735  0.004532   0.001133  0.006252  0.000000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-03            NaN  0.001152  0.001637  
2021-02-04            NaN  0.001151  0.001634  
2021-02-05            NaN  0.000000  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-03,0.039144,0.0,NaN,0.016016,0.009314,0.013890,0.002478,0.003365,0.019856,NaN,...,0.013434,0.011165,0.007122,0.017045,0.0,0.008900,0.016682,0.013294,0.002469,0.0
2021-02-04,0.019749,0.0,NaN,0.012964,0.004814,0.014970,0.003178,0.003912,0.004425,NaN,...,0.016844,0.010884,0.008250,0.011173,0.0,0.011407,0.006381,0.010690,0.007543,0.0
2021-02-05,0.010042,0.0,NaN,0.014252,0.008185,0.013219,0.004048,0.004036,0.049339,NaN,...,0.019898,0.011443,0.009936,0.000000,0.0,0.012331,0.007473,0.009615,0.004584,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount Bullock    Butler   
2021-02-03      0.014286  0.000000  0.050000  0.0  0.039604     0.0  0.000000   
2021-02-04      0.042254  0.021930  0.023810  0.0  0.038095     0.0  0.033333   
2021-02-05      0.027027  0.004292  0.023256  0.0  0.018349     0.0  0.000000   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2021-02-03      0.017021  0.025641      0.0  ...     0.0    0.0      0.0   
2021-02-04      0.012552  0.012500      0.0  ...     0.0    0.0      0.0   
2021-02-05      0.008264  0.000000      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-03          0.0        0.0   0.0   0.0        NaN      0.0  -0.20  
2021-02-04          0.0        0.0   0.0   0.0        NaN      0.0   0.25  
2021-02-05          0.0        0.0   0.0   0.0        NaN      0.0   0.00  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,0.000932,0.012108,0.002438,0.004890,0.002268,0.021128,0.004483,0.000932,0.000257,0.003315,...,0.004970,0.012629,0.000645,0.0,0.003952,0.028530,0.003002,0.000091,0.021377,0.005631
2021-02-04,0.000983,0.012553,0.002444,0.005772,0.002614,0.048717,0.004518,0.000905,0.000198,0.003472,...,0.005155,0.013129,0.000684,0.0,0.003768,0.016575,0.003561,0.000046,0.023856,0.005863
2021-02-05,0.000799,0.012917,0.002366,0.004612,0.002157,0.041149,0.004394,0.000924,0.000203,0.003389,...,0.005037,0.012260,0.000805,0.0,0.003655,0.013142,0.003246,0.000494,0.023995,0.005273


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,0.000581,0.005176,0.000969,0.007908,0.003528,0.005208,0.003074,0.001955,2.003292e-15,0.007176,...,0.011091,0.021372,4.204190e-07,NaN,0.005538,1.741564e-48,0.004208,5.972309e-07,0.016694,0.014159
2021-02-04,0.000498,0.007574,0.001174,0.003954,0.004956,0.002604,0.003196,0.002754,1.001646e-15,0.005739,...,0.009722,0.023617,2.102095e-07,NaN,0.005664,8.707818e-49,0.004258,2.986155e-07,0.013322,0.014565
2021-02-05,0.000249,0.009076,0.001448,0.006831,0.003535,0.001302,0.004524,0.001699,5.008231e-16,0.005201,...,0.009455,0.013909,1.051048e-07,NaN,0.005710,4.353909e-49,0.002665,1.493077e-07,0.016513,0.013106


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-03,0.002557,0.011049,0.002751,0.006854,0.005084,0.003768,0.004403,0.002444,0.000280,0.003640,...,0.010536,0.019386,0.000626,0.0,0.003944,0.001029,0.003557,0.000201,0.018556,0.018697
2021-02-04,0.001414,0.011499,0.002701,0.007829,0.003796,0.007471,0.004922,0.002331,0.000159,0.003334,...,0.009042,0.017718,0.000539,0.0,0.003802,0.001883,0.003169,0.000100,0.017460,0.016066
2021-02-05,0.000947,0.012572,0.002659,0.006522,0.003258,0.009260,0.005042,0.002124,0.000137,0.003450,...,0.008667,0.016290,0.000754,0.0,0.003766,0.000942,0.003597,0.000050,0.015942,0.014986


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-03,0.004515,0.003288,NaN,0.003928,0.006436,0.004149,0.002859,0.005632,0.004006,9.961014e-99,...,0.002765,0.008355,0.003857,0.010555,0.002694,0.005845,0.004714,0.004482,0.002308,0.002600
2021-02-04,0.005233,0.003331,NaN,0.004842,0.007327,0.004249,0.003413,0.004639,0.003541,4.980507e-99,...,0.003533,0.007013,0.003747,0.012334,0.003405,0.005902,0.004861,0.004585,0.002596,0.003021
2021-02-05,0.004215,0.003975,NaN,0.004900,0.006674,0.004610,0.004080,0.005093,0.003797,2.490253e-99,...,0.003574,0.006465,0.003604,0.010526,0.003547,0.007859,0.004779,0.004718,0.002492,0.002540


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-03      0.005766  0.005393  0.006339  0.002421  0.004123  0.003361   
2021-02-04      0.007924  0.004760  0.005188  0.002503  0.008476  0.008085   
2021-02-05      0.005080  0.004381  0.003851  0.002111  0.006720  0.006753   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-03      0.003874  0.005263  0.003945  0.003123  ...  0.001619   
2021-02-04      0.004385  0.005670  0.002900  0.003017  ...  0.000809   
2021-02-05      0.003817  0.005773  0.002994  0.001799  ... -0.000184   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-03      0.001310  0.002325  0.004366   0.002869  0.004311  0.002961   
2021-02-04     -0.001131  0.004605  0.004459   0.002855  0.008166  0.002238   
2021-02-05      0.001227  0.003670  0.004495   0.001994  0.007209  0.001119   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-03           -1.0  0.001314  0.001439  
2021-02-04           -1.0  0.001233  0.001536  
2021-02-05           -1.0  0.000616  0.000768  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-03,0.028178,0.016282,NaN,0.013328,0.008330,0.012649,0.002153,0.003621,0.013519,NaN,...,0.012972,0.009483,0.006945,0.011245,2.024614e-11,0.007138,0.009464,0.008077,0.003445,0.011836
2021-02-04,0.023964,0.008141,NaN,0.013146,0.006572,0.013810,0.002666,0.003767,0.008972,NaN,...,0.014908,0.010183,0.007597,0.011209,1.012307e-11,0.009273,0.007922,0.009383,0.005494,0.005918
2021-02-05,0.017003,0.004071,NaN,0.013699,0.007379,0.013514,0.003357,0.003901,0.029156,NaN,...,0.017403,0.010813,0.008767,0.005605,5.061536e-12,0.010802,0.007698,0.009499,0.005039,0.002959


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-03      0.011225  0.004983  0.025113  0.001349  0.024050  0.000070   
2021-02-04      0.026739  0.013456  0.024461  0.000674  0.031072  0.000035   
2021-02-05      0.026883  0.008874  0.023858  0.000337  0.024711  0.000017   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-03      0.000006  0.012124  0.019644  0.007751  ...  0.037725   
2021-02-04      0.016670  0.012338  0.016072  0.003875  ...  0.018863   
2021-02-05      0.008335  0.010301  0.008036  0.001938  ...  0.009431   

Province_State                                                         \
Admin2            Platte  Sheridan      Sublette Sweetwater     Teton   
2021-02-03      0.000197  0.039583  1.610412e-10   0.007953  0.042061   
2021-02-04      0.000099  0.019792  8.052060e-11   0.003976  0.021031   
2021-02-05      0.000049  0.009896  4.026030e-11   0.001988  0.010515   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2021-02-03      0.000026  -0.501961  0.010006 -0.037500  
2021-02-04      0.000013  -0.501961  0.005003  0.106250  
2021-02-05      0.000007  -0.501961  0.002501  0.053125  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210206,AK,53279,NaN,NaN,NaN,totalTestsViral,1536911,44.0,1219.0,...,53279,0,0,afd719917d160819ec0b42d3ef0d33b4649691e0,0,0,0,0,0,NaN
1,20210206,AL,471311,101101.0,1811811.0,NaN,totalTestsPeopleViral,2182021,1551.0,43005.0,...,2283122,64,0,67f81c52d48ece91b4357d95250cb495f4a5213a,0,0,0,0,0,NaN
2,20210206,AR,306064,62850.0,2277271.0,NaN,totalTestsViral,2520485,750.0,14049.0,...,2583335,11,147,e90bc100f78028a778010e5ba528095e37b7c1c6,0,0,0,0,0,NaN
3,20210206,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,4dcc829c2e6e71fe319092472376923951b194f2,0,0,0,0,0,NaN
4,20210206,AZ,779093,50485.0,2801489.0,NaN,totalTestsViral,6922180,3060.0,54507.0,...,3580582,63,198,3faacae1a6fd273b3d407b2a52676ffa816dd45c,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210206,AK,53279.0,NaN,NaN,NaN,totalTestsViral,1536911.0,44.0,1219.0,...,53279,0,0,afd719917d160819ec0b42d3ef0d33b4649691e0,0,0,0,0,0,NaN
1,20210206,AL,471311.0,101101.0,1811811.0,NaN,totalTestsPeopleViral,2182021.0,1551.0,43005.0,...,2283122,64,0,67f81c52d48ece91b4357d95250cb495f4a5213a,0,0,0,0,0,NaN
2,20210206,AR,306064.0,62850.0,2277271.0,NaN,totalTestsViral,2520485.0,750.0,14049.0,...,2583335,11,147,e90bc100f78028a778010e5ba528095e37b7c1c6,0,0,0,0,0,NaN
3,20210206,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,4dcc829c2e6e71fe319092472376923951b194f2,0,0,0,0,0,NaN
4,20210206,AZ,779093.0,50485.0,2801489.0,NaN,totalTestsViral,6922180.0,3060.0,54507.0,...,3580582,63,198,3faacae1a6fd273b3d407b2a52676ffa816dd45c,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-06,AK,53279,NaN,NaN,NaN,totalTestsViral,1536911,44.0,1219.0,NaN,...,53279,0,0,afd719917d160819ec0b42d3ef0d33b4649691e0,0,0,0,0,0,NaN
2021-02-06,AL,471311,101101.0,1811811.0,NaN,totalTestsPeopleViral,2182021,1551.0,43005.0,NaN,...,2283122,64,0,67f81c52d48ece91b4357d95250cb495f4a5213a,0,0,0,0,0,NaN
2021-02-06,AR,306064,62850.0,2277271.0,NaN,totalTestsViral,2520485,750.0,14049.0,263.0,...,2583335,11,147,e90bc100f78028a778010e5ba528095e37b7c1c6,0,0,0,0,0,NaN
2021-02-06,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,4dcc829c2e6e71fe319092472376923951b194f2,0,0,0,0,0,NaN
2021-02-06,AZ,779093,50485.0,2801489.0,NaN,totalTestsViral,6922180,3060.0,54507.0,849.0,...,3580582,63,198,3faacae1a6fd273b3d407b2a52676ffa816dd45c,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-06,AK,53279.0,NaN,NaN,NaN,totalTestsViral,1536911.0,44.0,1219.0,NaN,...,53279,0,0,afd719917d160819ec0b42d3ef0d33b4649691e0,0,0,0,0,0,NaN
2021-02-06,AL,471311.0,101101.0,1811811.0,NaN,totalTestsPeopleViral,2182021.0,1551.0,43005.0,NaN,...,2283122,64,0,67f81c52d48ece91b4357d95250cb495f4a5213a,0,0,0,0,0,NaN
2021-02-06,AR,306064.0,62850.0,2277271.0,NaN,totalTestsViral,2520485.0,750.0,14049.0,263.0,...,2583335,11,147,e90bc100f78028a778010e5ba528095e37b7c1c6,0,0,0,0,0,NaN
2021-02-06,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,4dcc829c2e6e71fe319092472376923951b194f2,0,0,0,0,0,NaN
2021-02-06,AZ,779093.0,50485.0,2801489.0,NaN,totalTestsViral,6922180.0,3060.0,54507.0,849.0,...,3580582,63,198,3faacae1a6fd273b3d407b2a52676ffa816dd45c,0,0,0,0,0,NaN
